In [1]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import keras
import pandas as pd
import numpy as np
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,roc_auc_score,f1_score

Using TensorFlow backend.


In [2]:
train= pd.read_csv('./nlp-getting-started/train.csv')
test=pd.read_csv('./nlp-getting-started/test.csv')
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, train.target.values, 
                                                  stratify=train.target.values, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)
xtest=test.text.values

print(xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape,xtest.shape)

(6851,) (762,) (6851,) (762,) (3263,)


In [9]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid) + list(xtest))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
xtest_seq = token.texts_to_sequences(xtest)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index


ytrain_enc = keras.utils.to_categorical(ytrain)
yvalid_enc = keras.utils.to_categorical(yvalid)

In [4]:
xtest_pad,word_index

(array([[    0,     0,     0, ...,  1591,   131,    97],
        [    0,     0,     0, ...,   664,  1596,   275],
        [    0,     0,     0, ...,   497,   100,    42],
        ...,
        [    0,     0,     0, ...,     1,     2, 29319],
        [    0,     0,     0, ...,     1,     2,  6123],
        [    0,     0,     0, ...,    75,   259,  3090]]),
 {'t': 1,
  'co': 2,
  'http': 3,
  'the': 4,
  'a': 5,
  'to': 6,
  'in': 7,
  'of': 8,
  'and': 9,
  'i': 10,
  'is': 11,
  'for': 12,
  'on': 13,
  'you': 14,
  'my': 15,
  'it': 16,
  'with': 17,
  'that': 18,
  'by': 19,
  'at': 20,
  'this': 21,
  'https': 22,
  'from': 23,
  'are': 24,
  'be': 25,
  'was': 26,
  'amp': 27,
  'have': 28,
  'like': 29,
  'as': 30,
  'up': 31,
  'just': 32,
  'me': 33,
  'but': 34,
  'not': 35,
  'so': 36,
  'your': 37,
  'out': 38,
  'no': 39,
  'will': 40,
  'after': 41,
  'all': 42,
  'fire': 43,
  'an': 44,
  'we': 45,
  'when': 46,
  'if': 47,
  "i'm": 48,
  'has': 49,
  'get': 50,
  'new': 51

In [5]:
# !pip install gensim

In [6]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [7]:
# dir(model)
model['king'].shape


(300,)

In [8]:
from tqdm import tqdm
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    try:
        embedding_vector = model[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue
    

100%|██████████████████████████████████████████████████████████████████████████| 29319/29319 [00:03<00:00, 7723.75it/s]


In [11]:
# model 1 LSTM
from keras.layers import SpatialDropout1D
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [ ]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 6851 samples, validate on 762 samples
Epoch 1/100
6851/6851 [==============================] - ETA: 29s - loss: 0.70 - ETA: 22s - loss: 0.69 - ETA: 19s - loss: 0.69 - ETA: 16s - loss: 0.68 - ETA: 14s - loss: 0.67 - ETA: 12s - loss: 0.67 - ETA: 10s - loss: 0.67 - ETA: 8s - loss: 0.6753 - ETA: 7s - loss: 0.672 - ETA: 5s - loss: 0.669 - ETA: 3s - loss: 0.666 - ETA: 2s - loss: 0.665 - ETA: 0s - loss: 0.664 - 22s 3ms/step - loss: 0.6630 - val_loss: 0.6100
Epoch 2/100
6851/6851 [==============================] - ETA: 18s - loss: 0.65 - ETA: 16s - loss: 0.65 - ETA: 15s - loss: 0.64 - ETA: 13s - loss: 0.63 - ETA: 12s - loss: 0.62 - ETA: 10s - loss: 0.61 - ETA: 9s - loss: 0.6139 - ETA: 7s - loss: 0.609 - ETA: 6s - loss: 0.604 - ETA: 5s - loss: 0.601 - ETA: 3s - loss: 0.597 - ETA: 2s - loss: 0.594 - ETA: 0s - loss: 0.590 - 21s 3ms/step - loss: 0.5894 - val_loss: 0.4622
Epoch 3/100
6851/6851 [==============================] - ETA: 17s - loss: 0.59 - ETA: 16s - loss: 0.56 - ETA: 16s - los

6851/6851 [==============================] - ETA: 17s - loss: 0.43 - ETA: 16s - loss: 0.42 - ETA: 15s - loss: 0.43 - ETA: 14s - loss: 0.43 - ETA: 12s - loss: 0.42 - ETA: 11s - loss: 0.42 - ETA: 9s - loss: 0.4223 - ETA: 8s - loss: 0.425 - ETA: 6s - loss: 0.427 - ETA: 5s - loss: 0.429 - ETA: 3s - loss: 0.427 - ETA: 2s - loss: 0.433 - ETA: 0s - loss: 0.434 - 22s 3ms/step - loss: 0.4347 - val_loss: 0.4172
Epoch 21/100
6851/6851 [==============================] - ETA: 20s - loss: 0.46 - ETA: 18s - loss: 0.44 - ETA: 16s - loss: 0.45 - ETA: 14s - loss: 0.45 - ETA: 13s - loss: 0.44 - ETA: 11s - loss: 0.44 - ETA: 10s - loss: 0.44 - ETA: 8s - loss: 0.4436 - ETA: 6s - loss: 0.439 - ETA: 5s - loss: 0.439 - ETA: 3s - loss: 0.439 - ETA: 2s - loss: 0.439 - ETA: 0s - loss: 0.439 - 22s 3ms/step - loss: 0.4409 - val_loss: 0.4058
Epoch 22/100
6851/6851 [==============================] - ETA: 18s - loss: 0.43 - ETA: 16s - loss: 0.42 - ETA: 14s - loss: 0.43 - ETA: 14s - loss: 0.44 - ETA: 12s - loss: 0.44 -

In [ ]:
from sklearn.metrics import roc_auc_score
predictions = model.predict(xvalid_pad)
# print(classification_report(predictions,yvalid))
score=roc_auc_score(yvalid,predictions[:,1])
print("{} score: {}".format("simpleLSTM",score))

In [ ]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model2 = Sequential()
model2.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model2.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.8))

model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.8))

model2.add(Dense(2))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model2.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

In [ ]:
predictions = model2.predict(xvalid_pad)
# print(classification_report(predictions,yvalid))
score=roc_auc_score(yvalid,predictions[:,1])
print("{} score: {}".format("BiLSTM",score))